# Automatic calibration of SWMM project 
#### Written by: Abullqasim Shakeri



## Import necessary packages and libraries


In [11]:
import pandas as pd
import random

from datetime import datetime
from datetime import timedelta

from swmm_api import read_out_file
from swmm_api.output_file.out import VARIABLES, OBJECTS

from pyswmm import Simulation, Links, Subcatchments


## Read and write some necessary files from the project folder

In [12]:
# Parameters and there ranges
ranges = pd.read_excel(r'Ranges.xlsx')


# Reading Observed Data
observed = pd.read_csv(r'Observed_flows.csv')
observed['date and time'] = pd.to_datetime(observed['Date'] + " " + observed['Time'], format='%m/%d/%Y %H:%M:%S')
observed['date and time2'] = pd.to_datetime(observed['Date'] + " " + observed['Time'], format='%m/%d/%Y %H:%M:%S')
observed.set_index("date and time2", inplace=True)


# Create a new data frame files in order to store new calibrated values
Calibration = pd.DataFrame(
    columns=["Rain_Events", "Subcatchment", "Imperviousness", "width", "Slope", "NSE_L1", "NSE_L2", "PFE_L1", "PFE_L2"])
Calibrated_sub = pd.DataFrame(
    columns=["Rain_Events", "Subcatchment", "Imperviousness", "width", "Slope", "NSE_L1", "NSE_L2", "PFE_L1", "PFE_L2"])
Calibrated_rain_event = pd.DataFrame(
    columns=["Rain_Events", "Subcatchment", "Imperviousness", "width", "Slope", "NSE_L1", "NSE_L2", "PFE_L1", "PFE_L2"])
Line = 55
Sub_N = 0
inde = 0

## Some new written functions

In [13]:
# To calculate NSE
def nse_f(calculated, observed_flow):
    numerator = (calculated - observed_flow) ** 2
    denominator = (observed_flow - observed_flow.mean()) ** 2
    nse = 1 - (numerator.sum() / denominator.sum())
    return nse



 # To calculate PFE
def pfe_f(calculated, observed_flow):
    pfe = abs((observed_flow.max() - calculated.max()) / observed_flow.max())
    return pfe




# To read out file and return value of link flow for the given out file. 
def read_out(name):
    out = read_out_file(name)
    outputfile = out.to_frame()
    pd.set_option("display.max_columns", None)

    link_1 = out.get_part(OBJECTS.LINK, '06L95-06L98', VARIABLES.LINK.FLOW_RATE).to_frame()
    link_2 = out.get_part(OBJECTS.LINK, '06M5-06M4', VARIABLES.LINK.FLOW_RATE).to_frame()
    link_1_2 = pd.concat([link_1, link_2], axis=1)
    link_flow = link_1_2[:End_date]
    return link_flow




# To assign new date for rain event
def time_assign(year, month, day):
    sim.step_advance(300)
    sim.start_time = datetime(year, month, day - 1, 0, 0, 0)
    sim.report_start = datetime(year, month, day, 0, 0, 0)
    sim.end_time = datetime(year, month, day + 1, 23, 55, 0)

    
    
    
# To run the model
def model_run():
    l1 = Links(sim)["06L95-06L98"]
    l2 = Links(sim)["06M5-06M4"]
    link_flow_df = pd.DataFrame(columns=["Date and time", "flow_L1", "flow_L2"])

    for step in sim:
        link_flow = sim.current_time, l1.flow
        link_flow = sim.current_time, l2.flow
        link_flow_df = link_flow_df.append({"Date and time": sim.current_time, "flow_L1": l1.flow, "flow_L2": l2.flow},
                                           ignore_index=True)

        
        
        
# To replace imperviousness value in the source .inp file
def par_replace(line_number_start, line_number_end):
    replaced_par = pd.DataFrame(
    columns=["Rain_Events", "Subcatchment", "Imperviousness", "width", "Slope", "NSE_L1", "NSE_L2", "PFE_L1", "PFE_L2"])
    Line = line_number_start
    Sub_N = 0
    while Line <= line_number_end:
        imper_n = pd.read_csv('imper.csv')
        Name_imper = imper_n["Name"][Sub_N]
        Rain_gage_imper = "RG"
        outlet_imper = imper_n["Outlet"][Sub_N]
        Area_imper = imper_n["Area"][Sub_N]
        Perc_imper = random.randint(ranges.at[Sub_N, "Imperviousness_1"], ranges.at[Sub_N, "Imperviousness_5"])
        Width_imper = random.randint(ranges.at[Sub_N, "Width_1"], ranges.at[Sub_N, "Width_5"])
        Perc_slope_imper = random.randint(ranges.at[Sub_N, "Slope_1"], ranges.at[Sub_N, "Slope_5"])
        Curblen_imper = "0"
        New_line = f"{Name_imper}             {Rain_gage_imper}               {outlet_imper}             {Area_imper}     {Perc_imper}          {Width_imper}      {Perc_slope_imper}     {Curblen_imper}\n"
        my_file = open("TUD_sewer_v9.inp")
        string_list = my_file.readlines()
        my_file.close()
        string_list[Line] = str(New_line)
        my_file = open("TUD_sewer_v9.inp", "w")
        new_file_contents = "".join(string_list)
        my_file.write(new_file_contents)
        my_file.close()
        readable_file = open("TUD_sewer_v9.inp")
        read_file = readable_file.read()
        Line = Line+1
        Sub_N = Sub_N +1
        New_data = {"Rain_Events": Start_date, "Subcatchment": Name_imper,
                "Imperviousness": Perc_imper,
                "width": Width_imper, "Slope": Perc_slope_imper, "NSE_L1": "NaN", "NSE_L2":"NaN",
                "PFE_L1":"NaN" ,
                "PFE_L2": "NaN"}
        replaced_par = replaced_par.append(New_data, ignore_index=True)

    return replaced_par


# Calibration process for all given rain event and catchments at once

In [14]:
# Create a new data frame files in order to store new calibrated values
Calibration = pd.DataFrame(
    columns=["Rain_Events", "Subcatchment", "Imperviousness", "width", "Slope", "NSE_L1", "NSE_L2", "PFE_L1", "PFE_L2"])

Rain_N = 0

while Rain_N <= 41:

    Sub_N = 0  # catchment

    wb = pd.read_excel(r'Dates_events.xlsx')
    year = wb.at[Rain_N, "year"]
    month = wb.at[Rain_N, "month"]
    day = wb.at[Rain_N, "Day"]

    print(year, month, day, 0, 0, 0)

    # Rain event from CSV
    rain_event = pd.read_csv(r'Dates_events_calibration.csv')
    rain_event["D and T"] = pd.to_datetime(rain_event["D and T"])

    # Select the observed data
    Start_date = rain_event.loc[Rain_N]["D and T"]
    End_date = Start_date + timedelta(days=1)
    after_start_date = observed["date and time"] >= Start_date
    before_end_date = observed["date and time"] <= End_date
    between_two_dates = after_start_date & before_end_date
    filtered_dates = observed.loc[between_two_dates]
    NSE_L1 = 0
    NSE_L2 = 0
    PFE_L1 = 1
    PFE_L2 = 1
    time_out = 0
    while True:

        New_data = par_replace(55, 73)
        with Simulation('TUD_sewer_v9.inp') as sim:

            time_assign(year, month, day)
            SubID = ranges.at[Sub_N, "Subcatchment"]
            Subca = Subcatchments(sim)[str(SubID)]
            model_run()
            Link_flow = read_out('TUD_sewer_v9.out')
            NSE_L1 = nse_f(Link_flow["link/06L95-06L98/Flow_rate"], filtered_dates["06L95-06L98"])
            PFE_L1 = pfe_f(Link_flow["link/06L95-06L98/Flow_rate"], filtered_dates["06L95-06L98"])
            NSE_L2 = nse_f(Link_flow["link/06M5-06M4/Flow_rate"], filtered_dates["06M5-06M4 "])
            PFE_L2 = pfe_f(Link_flow["link/06M5-06M4/Flow_rate"], filtered_dates["06M5-06M4 "])
        if NSE_L1 > 0.88 and NSE_L2> 0.88 and PFE_L2 < 0.15 and PFE_L1 < 0.15:
            Calibrated_sub = Calibrated_sub.append(New_data, ignore_index=True)
            break
        if time_out > 100:
            break
        time_out +=1
            
    New_data = {"Rain_Events": Start_date, "Subcatchment": "Name_imper", "Imperviousness": "Perc_imper", "width": "Width_imper", "Slope": "Perc_slope_imper", 
                            "NSE_L1": NSE_L1, "NSE_L2":NSE_L2, "PFE_L1":PFE_L1 , "PFE_L2": PFE_L2}
    Calibrated_sub = Calibrated_sub.append(New_data, ignore_index=True)
    Rain_N = Rain_N + 1
Calibration.to_csv('Calibration.csv')
Calibrated_sub.to_csv("Calibrated_sub.csv")   

1996 5 4 0 0 0


C:\Users\Abullqasim\anaconda3\lib\site-packages\swmm_api\output_file\extract.py:142: SwmmOutExtractWarning: Ending magic number incorrect.
  warn('Ending magic number incorrect.', SwmmOutExtractWarning)
C:\Users\Abullqasim\anaconda3\lib\site-packages\swmm_api\output_file\extract.py:239: SwmmOutExtractWarning: Infer time periods of the output file due to an corrupt SWMM .out-file.
  warn('Infer time periods of the output file due to an corrupt SWMM .out-file.', SwmmOutExtractWarning)


1996 6 3 0 0 0
1996 7 8 0 0 0
1996 8 11 0 0 0
1996 8 22 0 0 0
1997 7 18 0 0 0
1997 7 25 0 0 0
1998 6 3 0 0 0
1998 7 21 0 0 0
1998 9 12 0 0 0
1999 7 9 0 0 0
1999 7 14 0 0 0
1999 9 18 0 0 0
1999 10 9 0 0 0
2000 4 15 0 0 0
2001 7 7 0 0 0
2002 9 5 0 0 0
2003 5 9 0 0 0
2004 5 10 0 0 0
2004 9 12 0 0 0
2004 9 12 0 0 0
2004 11 9 0 0 0
2005 7 7 0 0 0
2005 8 22 0 0 0
2006 1 21 0 0 0
2006 5 12 0 0 0
2006 6 19 0 0 0
2007 6 16 0 0 0
2007 6 24 0 0 0
2008 8 8 0 0 0
2009 9 29 0 0 0
2010 6 2 0 0 0
2010 8 2 0 0 0
2011 7 3 0 0 0
2011 7 4 0 0 0
2013 10 10 0 0 0
2014 5 18 0 0 0
2014 5 26 0 0 0
2014 7 8 0 0 0
2014 9 19 0 0 0
2014 10 21 0 0 0
2015 8 17 0 0 0


## Final calibrated values for each rain events and their respective subcatchments
#### a .csv file of this result is created at project folder with the file name (Calibrated_sub.csv)

In [16]:
pd.set_option("display.max_rows", None, "display.max_columns", None)
Calibrated_sub


,Rain_Events,Subcatchment,Imperviousness,width,Slope,NSE_L1,NSE_L2,PFE_L1,PFE_L2
0,1996-05-04,101,77,408,3,NaN,NaN,NaN,NaN
1,1996-05-04,102,64,56,2,NaN,NaN,NaN,NaN
2,1996-05-04,103,28,128,4,NaN,NaN,NaN,NaN
3,1996-05-04,104,24,152,5,NaN,NaN,NaN,NaN
4,1996-05-04,105,21,211,8,NaN,NaN,NaN,NaN
5,1996-05-04,106,32,122,4,NaN,NaN,NaN,NaN
6,1996-05-04,107,36,53,3,NaN,NaN,NaN,NaN
7,1996-05-04,108,5,202,4,NaN,NaN,NaN,NaN
8,1996-05-04,109,40,226,5,NaN,NaN,NaN,NaN
9,1996-05-04,110,25,219,5,NaN,NaN,NaN,NaN
